<a href="https://colab.research.google.com/github/sagar3122/Time-Series/blob/master/TS_ARIMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
from datetime import datetime
import statsmodels.api as sm
from statsmodels.tsa.api import Holt, ExponentialSmoothing
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings("ignore")



train = pd.read_csv("Train_SU63ISt.csv")
test = pd.read_csv("Test_0qrQsBZ.csv")
#print(train)
train_original = train.copy()
test_original = test.copy()
# print(train.columns)
# print(test.columns)
# print(train.dtypes)
# print(test.dtypes)
# print(train.shape)
# print(test.shape)
train["Datetime"] = pd.to_datetime(train.Datetime, format = "%d-%m-%Y %H:%M")
test["Datetime"] = pd.to_datetime(test.Datetime, format = "%d-%m-%Y %H:%M")
test_original['Datetime'] = pd.to_datetime(test_original.Datetime,format='%d-%m-%Y %H:%M')
train_original['Datetime'] = pd.to_datetime(train_original.Datetime,format='%d-%m-%Y %H:%M')
# print(train.dtypes)
# print(test.dtypes)

for i in [train, test, train_original, test_original]:
    i["Year"] = i.Datetime.dt.year
    i["Month"] = i.Datetime.dt.month
    i["Day"] = i.Datetime.dt.day
    i["Hour"] = i.Datetime.dt.hour

#print(train.dtypes, test.dtypes)
#print(train)
train["day of week"] = train.Datetime.dt.dayofweek # .dayofweek attribute returns the number associated with the day, based on date, month and year(just like a calender)
train["day name"] = train.Datetime.dt.day_name() #day_name() returns the name of the day, based on date, month and year(just like a calender)
#temp = train["Datetime"] # temp is a series
#print(temp)
#print(train)

def weekend_not_weekend(row):
    if row.day_name() == "Saturday" or row.day_name() == "Sunday": # this if block does not work because day_name() is a datetime data type i am trying to match with a string data type
        return 1
    # elif row.dayofweek == 5 or row.dayofweek == 6:
    #     return 1
    else:
        return 0

temp2 = train["Datetime"].apply(weekend_not_weekend) #temp2 is a series # .apply() is a method in pandas used to apply a some other function over a axis of the data frame, could also be applied to a series/column of a data frame.
train["weekend"] = temp2 # series gets added to data frame with a new column name weekend
#print(temp2)
#print(train)

#train.index = train['Datetime'] # indexing the Datetime to the train df, so as to get the time period on the x-axis.

df = train.drop('ID',1)           # drop ID variable to get only the Datetime on x-axis.
#print(df)
ts = df['Count'] # Series of passenger count with index as the datetime.
#print(ts)
# plt.figure(figsize=(16,8)) #create a new figure with width 16 and height 8 inches.
# plt.plot(ts, label='Passenger Count')
# plt.title('Time Series')
# plt.xlabel("Time(year-month)")
# plt.ylabel("Passenger count")
# plt.legend(loc='best')
#plt.show()
#train.groupby("Year")["Count"].mean().plot(kind="bar")
# train = train.groupby("Year")["Count"].mean()
# print(train)
#train.groupby("Month")["Count"].mean().plot.bar()
# temp = train.groupby(["Year","Month"])["Count"].mean()
# temp.plot(figsize = (15,5), title = "Passsenger Count(MonthWise)", fontsize = "14")
#train.groupby("Day")["Count"].mean().plot.bar()
#train.groupby("Hour")["Count"].mean().plot.bar()
#train.groupby("weekend")["Count"].mean().plot.bar()
#train.groupby("day of week")["Count"].mean().plot.bar()
train = train.drop("ID",1)
#print(train)
#plt.show()
train.Timestamp = pd.to_datetime(train.Datetime,format = "%d-%m-%Y %H:%M")
#print(train)
train.index = train.Timestamp
hourly = train.resample("H")["Count"].mean()
#print(hourly)
daily = train.resample("D")["Count"].mean()
#print(daily)
weekly = train.resample("W")["Count"].mean()
#print(weekly)
monthly = train.resample("M")["Count"].mean()
#print(monthly)
#print(train)
# fig, axs = plt.subplots(4,1)
#
# hourly.plot(figsize=(15,8), title= 'Hourly', fontsize=14, ax=axs[0])
# daily.plot(figsize=(15,8), title= 'Daily', fontsize=14, ax=axs[1])
# weekly.plot(figsize=(15,8), title= 'Weekly', fontsize=14, ax=axs[2])
# monthly.plot(figsize=(15,8), title= 'Monthly', fontsize=14, ax=axs[3])

#plt.show()
# print(test)
test.Timestamp = pd.to_datetime(test.Datetime,format='%d-%m-%Y %H:%M')
test.index = test.Timestamp

# Converting to daily mean
#Taking the mean of all columns grouped by date
test = test.resample('D').mean()
# print(test)

train.Timestamp = pd.to_datetime(train.Datetime,format='%d-%m-%Y %H:%M')
train.index = train.Timestamp

# Converting to daily mean
#taking the mean of all columns grouped by date
train = train.resample('D').mean()

#splitting the data into training and validation part
Train = train.ix["2012-08-25":"2014-06-24"]
valid = train.ix["2014-06-25":"2014-09-25"]
# print(Train)
# print(valid)

#look how the train and validation parts have been divided
# Train.Count.plot(figsize=(15,8), title = "Daily Ridership", fontsize = 14, label = "train")
# valid.Count.plot(figsize=(15,8), title = "Daily Ridership", fontsize = 14, label = "valid")
# plt.xlabel("DateTime")
# plt.ylabel("Passenger Count")
# plt.legend(loc="best")
# plt.show()

#visualize the original time series data, trend in the time-series data, seasonality in the time-series data and the residual, from the training data set.
# result = sm.tsa.seasonal_decompose(Train.Count)
# print(result)
# sm.tsa.seasonal_decompose(Train.Count).plot()
# plt.show()
#adfuller test to check for unit root in the test series
#The Augmented Dickey-Fuller test can be used to test for a unit root in a univariate process in the presence of serial correlation.
result = sm.tsa.stattools.adfuller(train.Count)
# print(result)


#holt's linear trend Model
# hltm = valid.copy()
# fit1 = Holt(np.asarray(Train['Count'])).fit(smoothing_level = 0.3, smoothing_slope = 0.1)
# hltm['Holt_Linear'] = fit1.forecast(len(valid))
# # print(hltm)
# plt.figure(figsize = (16,8))
# plt.plot(Train['Count'], label = 'train')
# plt.plot(valid['Count'], label = 'valid')
# plt.plot(hltm['Holt_Linear'], label = 'holts_linear')
# plt.legend(loc = 'best')
# plt.show()

#holt's winter model
# hwm = valid.copy()
# fit_1 = ExponentialSmoothing(np.asarray(Train['Count']), seasonal_periods = 7, trend = 'add', seasonal = 'add').fit()
# hwm['Holts_winter_model'] = fit_1.forecast(len(valid))
# plt.figure(figsize = (16,8))
# plt.plot(Train['Count'], label = 'train')
# plt.plot(valid['Count'], label = 'valid')
# plt.plot(hwm['Holts_winter_model'], label = 'Holts_winter_model')
# plt.legend(loc = 'best')
# plt.show()

# print(train_original)
#ARIMA Model
#Dickey Fuller test for stationarity
def test_stationarity(timeseries):

    #Determing rolling statistics
    rolmean = timeseries.rolling(24).mean() # 24 hours on each day
    # print(rolmean)
    # print(type(rolmean))
    rolstd = timeseries.rolling(24).std()

    #Plot rolling statistics:
    orig = plt.plot(timeseries, color='blue',label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show()

    #Perform Dickey-Fuller test:
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    # print('dftest')
    # print(dftest)
    # print('break')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)

rcParams['figure.figsize'] = 20,10
test_stationarity(train_original['Count'])


#Removing the trend from the time series
#Apply log transformation
# print(Train.Count)
# print('break')
Train_log = np.log(Train['Count'])
# print(Train_log)
# print('break')
# print(np.exp(Train_log))
valid_log = np.log(valid['Count'])
#take rolling average to remove the trend
moving_avg = Train_log.rolling(24).mean()
# plt.plot(Train_log)
# plt.plot(moving_avg, color = 'r')
# plt.show()
#remove the increasing trend
# print(Train['Count'])
# print('break')
# print(Train_log)
# print('break')
# print(moving_avg)
# print('break')
train_log_moving_avg_diff = Train_log - moving_avg
# print(train_log_moving_avg_diff)
#drop the first 23 NaN values in the difference
train_log_moving_avg_diff.dropna(inplace = True)
# test_stationarity(train_log_moving_avg_diff)
#differencing to make the mean stable to make the series more stable and stationary and remove the trend
# print(Train_log)
# print('break')
# print(Train_log.shift(1))
train_log_diff = Train_log - Train_log.shift(1)
# print(train_log_diff)
#test_stationarity(train_log_diff.dropna())
#removing the seasonality
#print(pd.DataFrame(Train_log).Count.values)
#freq used in the next line overrides the periodicity of pandas object with the timeseries index
decomposition = seasonal_decompose(pd.DataFrame(Train_log).Count.values,freq = 24)#daily based seasonal decomposition
#used to access the 3 different components after seasonal decomposition
trend = decomposition.trend
# print(trend)
# print(type(trend))
# print("break")
seasonal = decomposition.seasonal
# print(seasonal)
# print("break")
residual = decomposition.resid
# print(residual)
# print("break")
#plotting the seasonal decompose
# plt.subplot(411)
# plt.plot(Train_log, label = 'Original')
# plt.legend(loc = 'best')
# plt.subplot(412)
# plt.plot(trend, label = 'trend')
# plt.legend(loc = 'best')
# plt.subplot(413)
# plt.plot(seasonal, label = 'seasonal')
# plt.legend(loc = 'best')
# plt.subplot(414)
# plt.plot(residual, label = 'residual')
# plt.legend(loc = 'best')
# plt.show()

#stationarity of the residuals
# train_log_decompose = pd.DataFrame(residual)
# #print(Train_log.index)
# train_log_decompose['date'] = Train_log.index
# train_log_decompose.set_index('date', inplace = True)
# # print(train_log_decompose)
# train_log_decompose.dropna(inplace = True)
# print(train_log_decompose[0])# providing a series for test
# test_stationarity(train_log_decompose[0])


#fitting the ARIMA model on the training data and then forcasting on the test data
# to fit ARIMA we need to find p,d,q parameters, for that we import acf and pacf form statsmodels.tsa.stattools
lag_acf = acf(train_log_diff.dropna(), nlags = 25)
lag_pacf = pacf(train_log_diff.dropna(), nlags = 25, method = 'ols')
#plot acf and pacf
# plt.plot(lag_acf)
# #adds a horizontal line
# plt.axhline(y=0,linestyle='--',color='gray')
# #lines to show the range of confidence interval
# plt.axhline(y=-1.96/np.sqrt(len(train_log_diff.dropna())),linestyle='--',color='gray')
# plt.axhline(y=1.96/np.sqrt(len(train_log_diff.dropna())),linestyle='--',color='gray')
# plt.title('Autocorrelation Function')
# plt.show()
# plt.plot(lag_pacf)
# plt.axhline(y=0,linestyle='--',color='gray')
# plt.axhline(y=-1.96/np.sqrt(len(train_log_diff.dropna())),linestyle='--',color='gray')
# plt.axhline(y=1.96/np.sqrt(len(train_log_diff.dropna())),linestyle='--',color='gray')
# plt.title('Partial Autocorrelation Function')
# plt.show()

#implementing the AR Model
#here the q value is 0 since its just the AR model
model = ARIMA(Train_log, order = (2, 1, 0))
# print(model)
results_AR = model.fit(disp = -1)
# print(result_AR)
# print(results_AR.fittedvalues)
# print(type(result_AR.fittedvalues))
# plt.plot(train_log_diff.dropna(), label = 'original')
# plt.plot(results_AR.fittedvalues, color = 'red', label = 'predictions' )
# plt.legend(loc = 'best')
# plt.show()
# we are doing code below to achieve the following points: plot the validation curve for AR model. the curve will a line plot showing the correctness of the prediction on the validation set.
# for that We have to change the scale of the model to the original scale.
# for that First step would be to store the predicted results as a separate series and observe it.
#we are doing this prediction for the validation set range
AR_predict = results_AR.predict(start = "2014-06-25", end = "2014-09-25")
# print(AR_predict)
AR_predict =  AR_predict.cumsum().shift().fillna(0) #don't know why we did that
# print(AR_predict)
# print('break')
# print(np.log(valid['Count'])[0])
AR_predict1 = pd.Series(np.ones(valid.shape[0])* np.log(valid['Count'])[0], index = valid.index) # why are we doing this?
# print(AR_predict1)
# print('break')
AR_predict1 =  AR_predict1.add(AR_predict, fill_value = 0) #why do we do this?
# print(AR_predict1)
# print('break')
AR_predict = np.exp(AR_predict1) #why do we do this?
# print(AR_predict)
#plot the predictions of the validation set range as a validation curve for AR model
# plt.plot(valid["Count"], label = 'valid')
# plt.plot(AR_predict, color = 'red', label = 'AR predictions')
# plt.legend(loc = 'best')
# plt.title('RMSE: %.4f' %(np.sqrt(np.dot(AR_predict, valid['Count']))/valid.shape[0])) # dont know why are we taking the dot product in mse computation
# #plt.title('RMSE: %.4f' %(np.sqrt(mean_squared_error(valid.Count, AR_predict))))
# plt.show()

#implement the MA Model
model = ARIMA(Train_log, order = (0, 1, 2))
results_MA = model.fit(disp = -1)
# plt.plot(train_log_diff.dropna(), label = 'original')
# plt.plot(results_MA.fittedvalues, color = 'r', label = 'predicted')
# plt.legend(loc = 'best')
# plt.show()
#plot the MA validation curve, follow the same steps as for AR validation curve
MA_predict=results_MA.predict(start="2014-06-25", end="2014-09-25")
MA_predict=MA_predict.cumsum().shift().fillna(0)
MA_predict1=pd.Series(np.ones(valid.shape[0]) * np.log(valid['Count'])[0], index = valid.index)
MA_predict1=MA_predict1.add(MA_predict,fill_value=0)
MA_predict = np.exp(MA_predict1)
#plot the validation curve now
# plt.plot(valid['Count'], label = "Valid")
# plt.plot(MA_predict, color = 'red', label = "Predict")
# plt.legend(loc= 'best')
# plt.title('RMSE: %.4f'% (np.sqrt(np.dot(MA_predict, valid['Count']))/valid.shape[0]))
# plt.show()

#combine both the models
# print(Train_log)
# print("break")
model = ARIMA(Train_log, order = (2,1,2))
results_ARIMA = model.fit(disp = -1)
# plt.plot(train_log_diff.dropna(), label = 'original')
# plt.plot(results_ARIMA.fittedvalues, color = 'red', label = 'prediected_ARIMA')
# plt.legend(loc = 'best')
# plt.show()

#plotting the validation curve like we did for both AR and MA models
#first lets predict the values for validation set using ARIMA model
ARIMA_predict_diff = results_ARIMA.predict(start="2014-06-25", end="2014-09-25")
# print(ARIMA_predict_diff)
# print('break')
# inorder to plot the validation curve for arima model, we need to first create a function which can be used to
# change the scale of the model to the original scale.
# what names of variables should have been
# ARIMA_predict_diff: ARIMA_prediction
# given_set: valid_set
#predict_diff: prediction
def check_prediction_diff(predict_diff, given_set):
    predict_diff = predict_diff.cumsum().shift().fillna(0)
    print(predict_diff)
    predict_base = pd.Series(np.ones(given_set.shape[0]) * np.log(given_set['Count'])[0], index = given_set.index)
    print(predict_base)
    #log transformation for predicted values
    predict_log = predict_base.add(predict_diff, fill_value = 0)
    # print(predict_log)
    predict = np.exp(predict_log)
    # print('break')
    # print(predict)

    #plot the curve
    # plt.plot(given_set['Count'], label = 'valid')
    # plt.plot(predict, color = 'red', label = 'prediction')
    # plt.plot(loc = 'best')
    # plt.title('RSME: %.4f' %(np.sqrt(np.dot(predict, given_set['Count']))/given_set.shape[0]))
    # print('RMSE:', np.sqrt(np.dot(predict, given_set['Count']))/given_set.shape[0])
    # plt.show()
# check_prediction_diff(ARIMA_predict_diff, valid)

#scale back the predictions to the original scale
def check_prediction_scale(predict, given_set):
    predict = predict.cumsum().shift().fillna(0)
    # print(predict)
    # print(valid.index.append(given_set.index))
    predict_base = pd.Series(np.ones(valid.shape[0] + given_set.shape[0]) * np.log(valid['Count'])[0], index = valid.index.append(given_set.index))
    # print(predict_base)
    #log transformation for predicted values
    predict_log = predict_base.add(predict, fill_value = 0)
    # print(predict_log)
    predict = np.exp(predict_log)
    # print('break')
    # print(predict)
    return predict

# print(test)
# print(np.log(valid['Count'])[0])
#predicting on test data using ARIMA
predict = results_ARIMA.predict(start = '2014-06-25', end = '2015-04-26')
# print(predict)
# check_prediction_scale(predict, test)
# print(check_prediction_scale(predict, test))



# Note that these are the daily predictions and we need hourly predictions.
# So, we will distribute this daily prediction into hourly counts.
# To do so, we will take the ratio of hourly distribution of passenger count from train data and
# then we will distribute the predictions in the same ratio.
test['prediction'] = check_prediction_scale(predict, test)
# Remember this is the daily predictions. We have to convert these predictions to hourly basis.
 # * To do so we will first calculate the ratio of passenger count for each hour of every day.
 # * Then we will find the average ratio of passenger count for every hour and we will get 24 ratios.
 # * Then to calculate the hourly predictions we will multiply the daily prediction with the hourly ratio.
 # every daily prediction is going to be multiplied by 24 such avg ratios of passenger count belonging to 24 hours in a day giving us the prediction of passenger count for each hour in a day.
#calculating the hourly ratio of count of passengers
# print(train_original)
train_original['ratio'] = train_original['Count']/train_original['Count'].sum()
# print(train_original)
#grouping the hourly ratio
temp = train_original.groupby(['Hour'])['ratio'].sum()
# print(temp)
#groupby to csv format
pd.DataFrame(temp, columns = ['Hour', 'ratio']).to_csv('Groupby.csv')
temp2 = pd.read_csv("Groupby.csv")
# print(temp2)
temp2 = temp2.drop("Hour.1",1)
# print(temp2)
#merge test and test_original on day, month and year
# print(test)
# print(test_original)
# test_original['Day'].astype(float)
# test_original['Month'].astype(float)
# test_original['Year'].astype(float)
# print(test.dtypes)
# print(test_original.dtypes)
merge = pd.merge(test, test_original, on=('Day', 'Month', 'Year'), how = 'left')
merge['Hour'] = merge['Hour_y']
merge = merge.drop(['Datetime', 'Hour_y', 'Hour_x', 'Year', 'Month'], axis = 1)
# print(merge)
#predicting by merging merge and temp2
# print(temp2)
prediction = pd.merge(merge, temp2, on = 'Hour', how = 'left')
# print(prediction)
# print(type(prediction))

#converting the ratio to the original scale
prediction['Count'] = prediction['prediction']*prediction['ratio']*24
prediction['ID'] = prediction['ID_y']
# print(prediction)

#lets drop all the features for the submission file, keep ID and Count only
submission=prediction.drop(['ID_x', 'Day', 'ID_y','prediction','Hour', 'ratio'],axis=1)
print("predictions made on the test data set\n")
print(submission)

# Converting the final submission to csv format
# pd.DataFrame(submission, columns=['ID','Count']).to_csv('ARIMA.csv')









#implementing SARIMA
# y_hat_avg = valid.copy()
# fit1 = sm.tsa.statespace.SARIMAX(Train.Count, order=(2, 1, 4),seasonal_order=(0,1,1,7)).fit()
# y_hat_avg['SARIMA'] = fit1.predict(start="2014-6-25", end="2014-9-25", dynamic=True)
# plt.figure(figsize=(16,8))
# plt.plot( Train['Count'], label='Train')
# plt.plot(valid['Count'], label='Valid')
# plt.plot(y_hat_avg['SARIMA'], label='SARIMA')
# plt.legend(loc='best')
# plt.show()
# predict=fit1.predict(start="2014-9-26", end="2015-4-26", dynamic=True)
# print(predict)
